# Converting SCT's Keras models into the .ONNX representation

## 1 sct_deepseg_sc models

Notes:
* In the "ctr" and "2d" cases, some code had to be copied and pasted from `core.py`, because the original Keras model-generating code wasn't well encapsulated.
* In the "3d" case, only a simple function call was necessary.
* When generating the ONNX model, opset 11 is needed because it corresponds to ir_version=6:
   * opset 15 and above (ir_version=8) isn't compatible with present versions of onnxruntime
   * opset 13/14 (ir_version=7) caused a warning that I forgot to document well
   * opset 12 (ir_version=7) caused a warning when converting the 3D model specifically

In [1]:
import os
import keras2onnx
from keras import backend as K
from spinalcordtoolbox.utils import sct_dir_local_path

TARGET_OPSET = 11
output_names = {}

Using TensorFlow backend.


### 1.1 Centerline detection models

In [2]:
from spinalcordtoolbox.deepseg_sc.cnn_models import nn_architecture_ctr

for contrast_type in ['t1', 't2', 't2s', 'dwi']:
    # NB: This is needed to reset the layer numbers, so that each model has identically named layers
    #     https://stackoverflow.com/questions/49123194/keras-reset-layer-numbers
    K.clear_session()
    
    # 1.1a Loading Keras model
    dct_patch_ctr = {'t2': {'size': (80, 80), 'mean': 51.1417, 'std': 57.4408},
                     't2s': {'size': (80, 80), 'mean': 68.8591, 'std': 71.4659},
                     't1': {'size': (80, 80), 'mean': 55.7359, 'std': 64.3149},
                     'dwi': {'size': (80, 80), 'mean': 55.744, 'std': 45.003}}
    dct_params_ctr = {'t2': {'features': 16, 'dilation_layers': 2},
                      't2s': {'features': 8, 'dilation_layers': 3},
                      't1': {'features': 24, 'dilation_layers': 3},
                      'dwi': {'features': 8, 'dilation_layers': 2}}
    ctr_model_fname = sct_dir_local_path('data', 'deepseg_sc_models', '{}_ctr.h5'.format(contrast_type))
    ctr_model = nn_architecture_ctr(height=dct_patch_ctr[contrast_type]['size'][0],
                                    width=dct_patch_ctr[contrast_type]['size'][1],
                                    channels=1,
                                    classes=1,
                                    features=dct_params_ctr[contrast_type]['features'],
                                    depth=2,
                                    temperature=1.0,
                                    padding='same',
                                    batchnorm=True,
                                    dropout=0.0,
                                    dilation_layers=dct_params_ctr[contrast_type]['dilation_layers'])
    ctr_model.load_weights(ctr_model_fname)
    # print(ctr_model.summary())
    
    # 1.1b Saving ONNX model
    ctr_model_fname_out = sct_dir_local_path('data', 'deepseg_sc_models', '{}_ctr.onnx'.format(contrast_type))
    if not os.path.isfile(ctr_model_fname_out):
        onnx_model = keras2onnx.convert_keras(ctr_model, f'ctr_model_{contrast_type}', target_opset=TARGET_OPSET)
        keras2onnx.save_model(onnx_model, ctr_model_fname_out)
        output_names[f"{contrast_type}_ctr"] = [n.name for n in onnx_model.graph.output]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



The ONNX operator number change on the optimization: 116 -> 33
The ONNX operator number change on the optimization: 108 -> 31
The ONNX operator number change on the optimization: 116 -> 33
The ONNX operator number change on the optimization: 108 -> 31


### 1.2 2D patch spinal cord segmentation models

In [3]:
from spinalcordtoolbox.deepseg_sc.cnn_models import nn_architecture_seg

kernel_size = '2d'
for contrast_type in ['t1', 't2', 't2s', 'dwi']:
    # NB: This is needed to reset the layer numbers, so that each model has identically named layers
    #     https://stackoverflow.com/questions/49123194/keras-reset-layer-numbers
    K.clear_session()
    
    # 1.2a Loading Keras model
    crop_size = 96 if (kernel_size == '3d' and contrast_type == 't2s') else 64  # Unnecessary, but preserved for posterity
    input_size = (crop_size, crop_size)
    seg_model_2d = nn_architecture_seg(height=input_size[0],
                                       width=input_size[1],
                                       depth=2 if contrast_type != 't2' else 3,
                                       features=32,
                                       batchnorm=False,
                                       dropout=0.0)
    model_fname_2d = sct_dir_local_path('data', 'deepseg_sc_models', '{}_sc.h5'.format(contrast_type))
    seg_model_2d.load_weights(model_fname_2d)
    # print(seg_model_2d.summary())
        
    # 1.2b Saving ONNX model
    model_fname_2d_out = sct_dir_local_path('data', 'deepseg_sc_models', '{}_sc.onnx'.format(contrast_type))
    if not os.path.isfile(model_fname_2d_out):
        onnx_model = keras2onnx.convert_keras(seg_model_2d, f'sc_2d_model_{contrast_type}', target_opset=TARGET_OPSET)
        keras2onnx.save_model(onnx_model, model_fname_2d_out)
        output_names[f"{contrast_type}_2d"] = [n.name for n in onnx_model.graph.output]           

The ONNX operator number change on the optimization: 78 -> 31
The ONNX operator number change on the optimization: 107 -> 42
The ONNX operator number change on the optimization: 78 -> 31
The ONNX operator number change on the optimization: 78 -> 31


### 1.3 3D patch spinal cord segmentation models

In [4]:
from spinalcordtoolbox.deepseg_sc.cnn_models_3d import load_trained_model

kernel_size = '3d'
for contrast_type in ['t1', 't2', 't2s']:  # NB: There is no 'dwi' 3D patch model (?)
    # NB: This is needed to reset the layer numbers, so that each model has identically named layers
    #     https://stackoverflow.com/questions/49123194/keras-reset-layer-numbers
    K.clear_session()
    
    # 1.3a Loading Keras model
    model_fname_3d = sct_dir_local_path('data', 'deepseg_sc_models', '{}_sc_3D.h5'.format(contrast_type))
    seg_model_3d = load_trained_model(model_fname_3d)
    # print(seg_model_3d.summary())
    
    # 1.3b Saving ONNX model
    model_fname_3d_out = sct_dir_local_path('data', 'deepseg_sc_models', '{}_sc_3D.onnx'.format(contrast_type))
    if not os.path.isfile(model_fname_3d_out):
        onnx_model = keras2onnx.convert_keras(seg_model_3d, f'sc_3d_model_{contrast_type}', target_opset=TARGET_OPSET)
        keras2onnx.save_model(onnx_model, model_fname_3d_out)
        output_names[f"{contrast_type}_3d"] = [n.name for n in onnx_model.graph.output]   

c:\users\joshua\repos\spinalcordtoolbox\venv_sct\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
The ONNX operator number change on the optimization: 42 -> 23
c:\users\joshua\repos\spinalcordtoolbox\venv_sct\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
The ONNX operator number change on the optimization: 42 -> 23
c:\users\joshua\repos\spinalcordtoolbox\venv_sct\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
The ONNX operator number chang

## 2. sct_deepseg_gm models

There is a slight snag with the sct_deepseg_gm models: The input layer doesn't have a fixed size; instead, it gets created on the fly based on the size of the input image:

```python
    ### Inside deepseg_gm.py :: segment_volume()
    if small_input:
        # Smaller than the trained net, don't crop
        net_input_size = volume_size
    else:
        # larger sizer, crop at 200x200
        net_input_size = (SMALL_INPUT_SIZE, SMALL_INPUT_SIZE)
    deepgmseg_model = model.create_model(metadata['filters'],
                                         net_input_size)

    ### Inside model.py :: create_model() 
    input_height, input_width = input_size
    inputs = Input((input_height, input_width, 1))
```

However, I think this _should_ be trivial to fix on the data side of things? e.g. We fix the input layer size to 200x200, then center + zero-pad any smaller input data to 200x200, then crop back to the original size afterwards.

That shouldn't change the inference results, no? (Since we use the same pretrained model weights regardless of input size, then input size _should_ have no bearing.) Hmmm...

In [5]:
import json

import spinalcordtoolbox.deepseg_gm.model as model
from spinalcordtoolbox.deepseg_gm.deepseg_gm import DataResource

for model_name in ['large', 'challenge']:
    # NB: This is needed to reset the layer numbers, so that each model has identically named layers
    #     https://stackoverflow.com/questions/49123194/keras-reset-layer-numbers
    K.clear_session()
    
    # 2a Loading Keras model
    gmseg_model_challenge = DataResource('deepseg_gm_models')
    model_path, metadata_path = model.MODELS[model_name]
    metadata_abs_path = gmseg_model_challenge.get_file_path(metadata_path)
    with open(metadata_abs_path) as fp:
        metadata = json.load(fp)
    deepgmseg_model = model.create_model(metadata['filters'])
    model_abs_path = gmseg_model_challenge.get_file_path(model_path)
    deepgmseg_model.load_weights(model_abs_path)
    # print(deepgmseg_model.summary())
    
    # 2b Saving ONNX model
    model_fname_gm_out = sct_dir_local_path('data', 'deepseg_gm_models', '{}_model.onnx'.format(model_name))
    if not os.path.isfile(model_fname_gm_out):
        onnx_model = keras2onnx.convert_keras(deepgmseg_model, f'gm_model_{model_name}', target_opset=TARGET_OPSET)
        keras2onnx.save_model(onnx_model, model_fname_gm_out)
        output_names[f"gm_{model_name}"] = [n.name for n in onnx_model.graph.output]   

The ONNX operator number change on the optimization: 141 -> 63
The ONNX operator number change on the optimization: 141 -> 63


## 3. sct_deepseg_lesion models

NB: sct_deepseg_lesion uses the same model type as the 3D patch version of sct_deepseg_sc? Huh! Interesting...

In [6]:
from spinalcordtoolbox.deepseg_sc.cnn_models_3d import load_trained_model

for contrast_type in ['t2', 't2_ax', 't2s']:
    # 3a Loading Keras model
    model_fname_lesion = sct_dir_local_path('data', 'deepseg_lesion_models', '{}_lesion.h5'.format(contrast_type))
    seg_model_lesion = load_trained_model(model_fname_lesion)
    # print(seg_model_lesion.summary())
    
    # 3b Saving ONNX model
    model_fname_lesion_out = sct_dir_local_path('data', 'deepseg_lesion_models', '{}_lesion.onnx'.format(contrast_type))
    if not os.path.isfile(model_fname_lesion_out):
        onnx_model = keras2onnx.convert_keras(seg_model_lesion, f'lesion_model_{contrast_type}', target_opset=TARGET_OPSET)
        keras2onnx.save_model(onnx_model, model_fname_lesion_out)
        output_names[f"{contrast_type}_lesion"] = [n.name for n in onnx_model.graph.output]   

c:\users\joshua\repos\spinalcordtoolbox\venv_sct\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
The ONNX operator number change on the optimization: 66 -> 38
c:\users\joshua\repos\spinalcordtoolbox\venv_sct\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
The ONNX operator number change on the optimization: 42 -> 23
c:\users\joshua\repos\spinalcordtoolbox\venv_sct\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
The ONNX operator number chang

## 4. Checking output channel names

The names of the output channels are needed when using the ONNX model to perform inference.

In [7]:
print(output_names)

{'t1_ctr': ['activation_12'], 't2_ctr': ['activation_11'], 't2s_ctr': ['activation_12'], 'dwi_ctr': ['activation_11'], 't1_2d': ['activation_11'], 't2_2d': ['activation_15'], 't2s_2d': ['activation_11'], 'dwi_2d': ['activation_11'], 't1_3d': ['activation_7'], 't2_3d': ['activation_7'], 't2s_3d': ['activation_7'], 'gm_large': ['predictions'], 'gm_challenge': ['predictions'], 't2_lesion': ['activation_11'], 't2_ax_lesion': ['activation_7'], 't2s_lesion': ['activation_11']}
